In [28]:
'''
Data collection script. Gathers businesses from yelp and google API searches
then scrapes the pages of the search results from both platforms to get all
review texts that include the word 'Halal' in them.
'''

from storage_managers.database import Database
import multiprocessing
from search_and_scrape.Google_business_search import get_google_places_by_location
from search_and_scrape.Yelp_business_search import get_yelp_places_by_location
from search_and_scrape.review_scraper import scrape_yelp_reviews, scrape_google_reviews, _close_webdriver
import sys
from datetime import datetime

def get_businesses():
    try:
        # searching for businesses and updating database
        coordinates = _get_coordinates_list()
        timestamped_print('Searching around', '|'.join([coord[0] for coord in coordinates]))
        agents = 3
        chunksize = 10
        businesses_list = []
        # with multiprocessing.Pool(processes=agents) as pool:
        #     businesses_list.extend(pool.map(get_google_places_by_location, coordinates, chunksize))
        #     # businesses_list.extend(pool.map(get_yelp_places_by_location, coordinates, chunksize))
        #     pool.close()
        #     pool.join()
        ## -Chunk and call search functions multiple times:
        ## -Have lat,lng limitation where if more than 10% of results are outside locality then stop search
        ## and eliminate results from lat/lng +/- threshold
        _update_businesses(businesses_list)
    except KeyboardInterrupt:
        _update_businesses(businesses_list)
        raise
        sys.exit(0)
        
def scrape_reviews():
    yelp_urls, google_urls = _get_unscraped_urls()
    # scrape reviews info
    agents = 3
    chunksize = 10
    reviews_list = []
    print('\n#####################################################################')
    print('{} yelp restaurants and {} Google restaurants left to scrape'.format(len(yelp_urls), len(google_urls)))
    try:
        with multiprocessing.Pool(processes=agents) as pool:
            reviews_list.extend(pool.starmap(scrape_yelp_reviews, yelp_urls[:3], chunksize=chunksize))
            # reviews_list.extend(pool.starmap(scrape_google_reviews, google_urls[:2], chunksize))
            pool.close()
            pool.join()
        _update_reviews(reviews_list)
    except (KeyboardInterrupt, SystemExit):
        _update_reviews(reviews_list)
        raise
        sys.exit(0)


def _get_unscraped_urls():
    db = Database()
    # get list of google and yelp urls
    get_urls = '''SELECT url, platform_id
                    FROM businesses
                    WHERE url LIKE %s '''
    yelp_urls = db.select_rows(get_urls, ('%yelp%', ))
    google_urls = db.select_rows(get_urls, ('%google%', ))

    # exclude businesses that have already been scraped
    scraped_ids = db.select_rows('''SELECT DISTINCT ON (restaurant_id) restaurant_id FROM reviews''')
    exclusion_list = [item[0] for item in scraped_ids]
    yelp_urls_keep = [t for t in yelp_urls if t[1] not in exclusion_list]
    google_urls_keep = [t for t in google_urls if t[1] not in exclusion_list]
    return yelp_urls_keep, google_urls_keep


def _update_reviews(reviews_list):
    '''
        get a nested list of individual review data. Each entry includes:
        - restaurant id
        - username
        - rating
        - review text
        - review date
        - helpful count
    '''
    db = Database()
    # update database with business scraping results
    reviews_sql = """INSERT INTO reviews (restaurant_id, username, rating, review_text, date, helpful_count)
                    VALUES (%s, %s, %s, %s, %s, %s )
                    ON CONFLICT (review_text) DO NOTHING"""
    db_list = [item for sublist in reviews_list for item in sublist]
    db.insert_rows(reviews_sql, *db_list)
    #print summary statement
    timestamped_print('Attempted to insert {} reviews'.format(len(db_list)))

def timestamped_print(*args, **kwargs):
  print(datetime.now(), *args, **kwargs)

In [32]:
reviews_list = scrape_reviews()


#####################################################################
3260 yelp restaurants and 98 Google restaurants left to scrape


In [33]:
db_list = [item for sublist in reviews_list for item in sublist]
len(db_list)

4

In [34]:
_update_reviews(reviews_list)

2021-01-19 13:53:19.865681 Attempted to insert 4 reviews


---

In [35]:
from main import _update_reviews

_update_reviews([['aP_9qqZQKTE3lLUIbZt1UQ', '', '', '', '', 0]])

TypeError: not all arguments converted during string formatting

In [73]:
reviews_list = [[['aP_9qqZQKTE3lLUIbZt1UQ', '', '', '', '', 0]], [['A-zhHGnBFkKA9fo1neN7iA', '', '', '', '', 0]]]
reviews_sql = """INSERT INTO reviews (restaurant_id, username, rating, review_text, date, helpful_count)
                    VALUES (%s, %s, %s, %s, %s, %s )
                    ON CONFLICT (review_text) DO NOTHING"""
db_list = [item for sublist in reviews_list for item in sublist]
for review in db_list:
    for r in (('_', '\_'), ('-', '\-')):
        review[0] = review[0].replace(*r)
    db.insert_row(reviews_sql, *review)

In [104]:
from storage_managers.database import Database
db = Database()
reviews_list = [[['aP_9qqZQKTE3lLUIbZt1UQ', '', '', '', '', 0]], [['A-zhHGnBFkKA9fo1neN7iA', '', '', '', '', 0]]]
# update database with business scraping results
reviews_sql = """INSERT INTO reviews (restaurant_id, username, rating, review_text, date, helpful_count)
                VALUES (%s, %s, %s, %s, %s, %s )
                ON CONFLICT (review_text) DO NOTHING"""
db_list = [item for sublist in reviews_list for item in sublist]
for review in db_list:
    if review[3] == '': #for restaurants without reviews
        review[3] = None #add restaurant_id to end of list again
        print(review)
    db.insert_row(reviews_sql, *review)

['aP_9qqZQKTE3lLUIbZt1UQ', '', '', None, '', 0]
['A-zhHGnBFkKA9fo1neN7iA', '', '', None, '', 0]
